In [18]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import os
import random
import warnings
warnings.filterwarnings('ignore')



In [19]:

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)



In [20]:
from keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(42)
sess = tf.Session(config=config)
K.set_session(sess)



In [21]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [22]:
data = pd.read_csv('epl_train.csv')

# Preview data.
data = data[data.MW > 3]

data.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

In [23]:

cleanup_nums = {"FTR":     {"H": 2, "D": 1, "A": 0}}

data = data.replace(cleanup_nums)

In [24]:
X_all = data.drop(['FTR'],1)
y_all = data['FTR']

In [25]:
#last 3 wins for both sides
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [26]:
X_all.head()

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.00,1.50,1,0,0,0,1,0,0,0,...,0,0,1,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,2.25,1.00,0,0,1,0,0,1,0,0,...,0,1,0,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,0.75,0.75,0,1,0,0,0,1,0,1,...,0,1,0,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,1.50,0.75,0,0,1,0,0,1,0,1,...,0,0,1,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0.75,0.75,0,0,1,0,1,0,0,1,...,0,0,1,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [27]:
from sklearn.model_selection import train_test_split



# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.1,
                                                    random_state = 2,
                                                    stratify = y_all)


In [28]:
# input dimension is number of features
input_dim = X_train.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
learning_rate = 0.001
batch_size = 16

model = keras.Sequential([
    Dense(48, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.3),
    Dense(16),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

In [29]:
es = EarlyStopping(monitor='loss', patience=3, verbose=1)

In [30]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam



opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
X_train.head()

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
2817,1.687500,0.750000,0,1,0,0,0,1,0,1,...,1,0,0,1.22,7.0,15.00,0.812500,-1.125000,0.125000,-16.0
478,1.100000,1.200000,0,0,1,1,0,0,0,1,...,1,0,0,2.20,3.2,3.50,0.100000,0.100000,0.000000,2.0
1894,2.184211,1.052632,0,0,1,0,0,1,0,0,...,0,1,0,1.17,9.0,17.00,1.657895,-0.236842,0.263158,-12.0
2439,2.000000,0.937500,0,0,1,1,0,0,0,0,...,0,0,1,3.20,3.5,2.35,0.687500,-0.437500,0.562500,-9.0
1786,1.370370,1.037037,0,1,0,0,1,0,0,1,...,0,1,0,2.10,3.4,3.90,-0.222222,-0.333333,-0.037037,-5.0


In [32]:
model.fit(X_train, y_train, batch_size=batch_size, 
              callbacks=[es], epochs=500, verbose=0)

Epoch 13: early stopping


In [33]:
train_loss, train_acc = model.evaluate(X_train, y_train)
val_loss, val_acc = model.evaluate(X_test, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Validation loss:', val_loss)
print('Validation accuracy:', val_acc)

Training loss: 0.9106119689434835
Training accuracy: 0.5736143
Validation loss: 0.9126245625607379
Validation accuracy: 0.5792208


In [34]:
labels = ['Home', 'Draw', 'Away']
y_preds = model.predict(X_test)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax, target_names=labels))
print(confusion_matrix(y_test, y_pred_argmax))

              precision    recall  f1-score   support

        Home       0.57      0.62      0.59       111
        Draw       0.00      0.00      0.00        94
        Away       0.58      0.86      0.69       180

    accuracy                           0.58       385
   macro avg       0.38      0.49      0.43       385
weighted avg       0.44      0.58      0.50       385

[[ 69   0  42]
 [ 26   0  68]
 [ 26   0 154]]
